**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Notebook 3 - Open Science Platform - Project Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a project account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the project account;

6 - Queries the blockchain again to confirm the succesfull details setting.

Every account succesfully created is dumped to `datasets/projects.json` for later use and evidence collection.  

In [1]:
# from name_generator import left, right
from utilities import *
from project_id_generator import * # - Random generated synthetic project ID
from iroha_helper import *
from loguru import logger
from dump_to_json import ProjectAccount, update_or_append_project_metadata
from ipfs_functions import download_json_from_ipfs
from synthetic_project_json_ld import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [2]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.  

In [3]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-02-09 10:20:49,214 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-02-09 10:20:49.215 | INFO     | __main__:<module>:3 - b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'


2 - Creates a project account into the blockchain;

In [4]:
project_private_key = IrohaCrypto.private_key()
project_public_key = IrohaCrypto.derive_public_key(project_private_key).decode("utf-8")

project_id = generate_project_id()

logger.info(f"Creating project with name: {project_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_project_account(create_account_contract_address, project_id, DOMAIN, project_public_key)


2025-02-09 10:20:49.277 | INFO     | __main__:<module>:6 - Creating project with name: 87810
2025-02-09 10:20:49.280 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 10:20:49.302 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'a76bb4a33334ef018daf965042478cfc41229121be8e4d461a3dc9f9c709c2ce'
2025-02-09 10:20:49.304 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:20:49.305 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 10:20:49.306 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_project_account"
2025-02-09 10:20:49,323 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:20:49,324 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:20:50,883 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:20

3 - Queries the blockchain to confirm the succesful creation of the account;

In [11]:
#Query - GetAccountDetail

# - Get account info
project_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(project_account_hash)


2025-02-09 10:20:50.917 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 10:20:50,919 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:20:50,936 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:20:50,938 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:20:53,910 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:20:53,918 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:20:53,920 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': 'f2ee3c85ce3d588fd388f00bd36134430df39351e7259d71f97f77e3d8e0dd3f', 'timestamp': 1739096453920}
2025-02-09 10:20:53.921 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 10:20:53,924 - INFO - Appended data under '87810@test' in logs/account_data.json
2025-02-09 10:20:53.926 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 10:20:53.927

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [6]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-02-09 10:20:53,967 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-02-09 10:20:53.968 | INFO     | __main__:<module>:3 - b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'


5 - Sets details for the project account:

In [7]:
# Generate synthetic project metadata JSON (assumed to be done already)
project_metadata_json = generate_synthetic_entry_ld()
account_metadata_cid = upload_json_to_ipfs(project_metadata_json)  # Assuming this is done before calling the function

# Call the function to update or append a new entry to the JSON-LD file
updated_entry_count = update_or_append_project_metadata(
    project_id, 
    project_metadata_json, 
    account_metadata_cid
)


address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

# Setting account detail using the generated project metadata CID
hash = set_account_detail(
    address, 
    f"{project_id}@{DOMAIN}",  # Project ID as account
    "account_metadata_cid",    # The key we're setting
    account_metadata_cid       # The value (CID from IPFS)
)

2025-02-09 10:20:54.058 | INFO     | dump_to_json:update_or_append_project_metadata:216 - Updated existing project entry for project ID: 87810@test
2025-02-09 10:20:54.061 | INFO     | dump_to_json:update_or_append_project_metadata:235 - Current total entries in 'datasets/projects.json': 2
2025-02-09 10:20:54.063 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-02-09 10:20:54.085 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'86452d50ed5d8cdb132dc84c548608c8454a8a993a6deab3384e17e1aee55ec0'
2025-02-09 10:20:54.087 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: A518E3EE4D45C20EF834361093305D2500A46228
2025-02-09 10:20:54.088 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-02-09 10:20:54.089 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"


2025-02-09 10:20:54,104 - INFO - None
2025-02-09 10:20:54,108 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:20:56,942 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:20:56,950 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:20:56.952 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-02-09 10:20:56.972 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 000000000000000000000000000000000000000000000000000000000000002000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000001
2025-02-09 10:20:56.973 | DEBUG    | integration_helpers:get_engine_receipts_result:140 - Result decoded string:                                                                                                
2025-02-09 10:20:56.974 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_result"
2025-02-09 10:20:56.975 | DEBUG    | integrati

6 - Queries the blockchain to confirm the succesful details setting

In [8]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(details) #TO DO - From now

2025-02-09 10:20:56.995 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-02-09 10:20:56,997 - WARNING - create_account_contract_address: 93F622673AA712B7298FDD2212E3228D8D9DD8BF
2025-02-09 10:20:57,013 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-02-09 10:20:57,015 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-02-09 10:20:59,970 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-02-09 10:20:59,978 - INFO - ('COMMITTED', 5, 0)
2025-02-09 10:20:59,980 - WARNING - Account Data: {'address': '93F622673AA712B7298FDD2212E3228D8D9DD8BF', 'hex_hash': '28161721be5875d176143241df2076a66fcb0c1b9e10bcced4b4a15347fdc11f', 'timestamp': 1739096459980}
2025-02-09 10:20:59.982 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-02-09 10:20:59,985 - INFO - Appended data under '87810@test' in logs/account_data.json
2025-02-09 10:20:59.987 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-02-09 10:20:59.988

7 - Retrieves Project metadata (JSON-LD formatted) from IPFS

In [9]:
project_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {project_metadata}")

2025-02-09 10:21:00.042 | INFO     | __main__:<module>:2 - User Metadata: {'@context': {'dc': 'http://purl.org/dc/terms/', 'schema': 'http://schema.org/'}, '@type': 'schema:ResearchProject', 'dc:abstract': 'This paper analyzes how big data influences precision medicine, providing insights into how to maximize its environmental sustainability.', 'schema:endDate': '2027-10-17', 'schema:funding': {'@type': 'schema:Organization', 'schema:name': 'United Nations Environment Programme'}, 'schema:keywords': ['big data', 'precision medicine', 'environmental sustainability'], 'schema:location': {'@type': 'schema:Place', 'schema:name': 'Rotterdam, Netherlands'}, 'schema:name': 'Analyzing the Influence of big data on precision medicine', 'schema:startDate': '2024-07-26'}
